In [1]:
%pwd

'd:\\ITI\\DS Track\\Deep Learning\\Projects\\Kidney Classification\\Kidney-Disease-Classification\\research'

In [2]:
import os

os.chdir("../")

%pwd

'd:\\ITI\\DS Track\\Deep Learning\\Projects\\Kidney Classification\\Kidney-Disease-Classification'

## Workflows

1. Update config.yaml
2. Update secrets.yaml [Optional]
3. Update params.yaml
4. Update the entity
5. Update the configuration manager in src config
6. Update the components
7. Update the pipeline 
8. Update the main.py
9. Update the dvc.yaml
10. app.py

# Update Entity

In [3]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class TrainingConfig:
    root_dir:Path
    trained_model_path:Path
    updated_base_model:Path
    training_data:Path
    params_epochs:int
    params_batch_size:int
    params_is_augmented:bool
    params_image_size:list



# Update ConfigManger

In [5]:
from src.CNNClassifierKidneyDiseases.constants import *
from src.CNNClassifierKidneyDiseases.utils.common import read_yaml , create_directories
import torch
import os
from zipfile import ZipFile
import urllib.request as requests
import timm
import torch.nn as nn
from torch.nn import Module
import torch.optim as optim
from typing import Optional
from pathlib import Path
import tqdm 





In [26]:
class ConfigurationManager:
    # THOSE CONSTATN VARS, from src.constants 
    def __init__(self, config_filepath = CONFIG_FILE_PATH, params_filepath =PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifact_root])

    
    def get_training_config(self)->TrainingConfig:

        training = self.config.training
        prepare_base_model = self.config.prepare_base_model
        params = self.params
        training_data = os.path.join(self.config.data_ingestion.unzip_dir,"kidney-ct-scan-image")

        create_directories([training.root_dir])


        training_config = TrainingConfig(

            root_dir=training.root_dir,
            trained_model_path=Path(training.training_model_path),
            updated_base_model=Path(prepare_base_model.updated_base_model_path),
            training_data=Path(training_data),
            params_epochs=params.EPOCHS,
            params_batch_size=params.BATCH_SIZE,
            params_is_augmented=params.AUGMENTATION,
            params_image_size=params.IMAGE_SIZE

        )

        return training_config

    

# Updata Components

In [8]:
import os
from zipfile import ZipFile
import urllib.request as requests
import timm
import torch.nn as nn
from torch.nn import Module
import torch.optim as optim
from typing import Optional
from pathlib import Path
import tqdm 


from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader

In [40]:
class Training:

    def __init__(self,config:TrainingConfig):
        self.config = config
    
    def get_base_model(self):
        self.model = torch.load(self.config.updated_base_model,weights_only=False)
        #self.model.eval()

    def train_valid_loader(self):
        image_size = self.config.params_image_size[1:]
        means = [0.1921, 0.1921, 0.1921]
        stds = [0.2601, 0.2601, 0.2601]

        if self.config.params_is_augmented:
            train_transform = transforms.Compose([

                transforms.RandomResizedCrop(image_size),
                transforms.RandomHorizontalFlip(),
                transforms.RandomRotation(degrees=30),
                transforms.ToTensor(),
                transforms.Normalize(mean = means , std = stds)
            ])
        else:
            train_transform = transforms.Compose([

                transforms.Resize(image_size),
                transforms.ToTensor(),
                transforms.Normalize(mean=means , std=stds)
            ])

            valid_transforms = transforms.Compose([

                transforms.Resize(image_size),
                transforms.ToTensor(),
                transforms.Normalize(mean=means, std= stds)
            ])

        # full dataset

        full_dataset = datasets.ImageFolder(self.config.training_data,transform=train_transform)
        #self.train_dataset, self.val_dataset = random_split(full_dataset,[0.8 , 0.2], generator=g)
        val_size = int(0.2 * len(full_dataset))
        train_size = len(full_dataset) - val_size

        # Splitting the train valid
        self.train_dataset , self.valid_dataset = torch.utils.data.random_split(full_dataset,[train_size , val_size])


        # Train loader And Valid loader

        self.train_loader = DataLoader(self.train_dataset , batch_size=self.config.params_batch_size , shuffle=True)

        self.valid_loader = DataLoader(self.valid_dataset, batch_size=self.config.params_batch_size, shuffle=False)

    @staticmethod
    def save_model(path : Path , model: torch.nn.Module):
        torch.save(model,path)

    def train(self):
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        self.model = self.model.to(device)
        criteration = nn.CrossEntropyLoss()
        optimizer = optim.Adam(self.model.parameters() , lr = 0.001)

        for epoch in range(self.config.params_epochs):
            self.model.train()
            total_loss , correct , total = 0,0,0

            for images , labels in self.train_loader:
                images , labels = images.to(device) , labels.to(device)

                optimizer.zero_grad()
                y_pred = self.model(images)
                loss = criteration(y_pred , labels)
                loss.backward()

                optimizer.step()

                total_loss += loss.item()
                _, preds = torch.max(y_pred,1)
                correct += (preds ==labels).sum().item()
                total += labels.size(0)
        train_accuracy = correct / total
        print(f"Epoch {epoch+1}/{self.config.params_epochs}, "
                f"Loss: {total_loss:.4f}, Accuracy: {train_accuracy:.4f}")
        

        # Validation
        self.model.eval()
        val_correct, val_total = 0, 0
        with torch.no_grad():
            for images, labels in self.valid_loader:
                images, labels = images.to(device), labels.to(device)
                outputs = self.model(images)
                _, preds = torch.max(outputs, 1)
                val_correct += (preds == labels).sum().item()
                val_total += labels.size(0)

        val_accuracy = val_correct / val_total
        print(f"Validation Accuracy: {val_accuracy:.4f}")

        self.save_model(path=self.config.trained_model_path, model=self.model)

In [ ]:
from src.CNNClassifierKidneyDiseases import logger

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from pathlib import Path

class Training:

    def __init__(self, config: TrainingConfig):
        self.config = config

    def get_base_model(self):
        self.model = torch.load(self.config.updated_base_model, weights_only=False)

    def train_valid_loader(self):
        image_size = self.config.params_image_size[1:]
        means = [0.1921, 0.1921, 0.1921]
        stds = [0.2601, 0.2601, 0.2601]

        if self.config.params_is_augmented:
            train_transform = transforms.Compose([
                transforms.RandomResizedCrop(image_size),
                transforms.RandomHorizontalFlip(),
                transforms.RandomRotation(degrees=30),
                transforms.ToTensor(),
                transforms.Normalize(mean=means, std=stds)
            ])
        else:
            train_transform = transforms.Compose([
                transforms.Resize(image_size),
                transforms.ToTensor(),
                transforms.Normalize(mean=means, std=stds)
            ])

        valid_transforms = transforms.Compose([
            transforms.Resize(image_size),
            transforms.ToTensor(),
            transforms.Normalize(mean=means, std=stds)
        ])

        full_dataset = datasets.ImageFolder(self.config.training_data, transform=train_transform)
        val_size = int(0.2 * len(full_dataset))
        train_size = len(full_dataset) - val_size

        self.train_dataset, self.valid_dataset = torch.utils.data.random_split(full_dataset, [train_size, val_size])
        self.train_loader = DataLoader(self.train_dataset, batch_size=self.config.params_batch_size, shuffle=True)
        self.valid_loader = DataLoader(self.valid_dataset, batch_size=self.config.params_batch_size, shuffle=False)

    @staticmethod
    def save_model(path: Path, model: torch.nn.Module):
        torch.save(model, path)

    def train(self):
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        self.model = self.model.to(device)
        criterion = nn.CrossEntropyLoss()
        optimizer = optim.Adam(self.model.parameters(), lr=0.001)

        logger.info(f"Training started on device: {device}")

        for epoch in range(self.config.params_epochs):
            self.model.train()
            total_loss, correct, total = 0, 0, 0

            for batch_idx, (images, labels) in enumerate(self.train_loader):
                images, labels = images.to(device), labels.to(device)
                optimizer.zero_grad()
                y_pred = self.model(images)
                loss = criterion(y_pred, labels)
                loss.backward()
                optimizer.step()

                total_loss += loss.item()
                _, preds = torch.max(y_pred, 1)
                correct += (preds == labels).sum().item()
                total += labels.size(0)

                # Log loss for each batch
                """logger.info(f"Epoch [{epoch+1}/{self.config.params_epochs}] - "
                            f"Batch [{batch_idx+1}/{len(self.train_loader)}] - "
                            f"Batch Loss: {loss.item():.4f}") """

            train_accuracy = correct / total
            logger.info(f"Epoch [{epoch+1}/{self.config.params_epochs}] - "
                        f"Total Loss: {total_loss:.4f} - Accuracy: {train_accuracy:.4f}")

        # Validation
        self.model.eval()
        val_correct, val_total = 0, 0
        with torch.no_grad():
            for images, labels in self.valid_loader:
                images, labels = images.to(device), labels.to(device)
                outputs = self.model(images)
                _, preds = torch.max(outputs, 1)
                val_correct += (preds == labels).sum().item()
                val_total += labels.size(0)

        val_accuracy = val_correct / val_total
        logger.info(f"Validation Accuracy: {val_accuracy:.4f}")

        self.save_model(path=self.config.trained_model_path, model=self.model)
        logger.info(f"Model saved to: {self.config.trained_model_path}")


In [46]:
# Run
try:
    config = ConfigurationManager()
    training_config = config.get_training_config()
    training = Training(config=training_config)
    training.get_base_model()
    training.train_valid_loader()
    training.train()

except Exception as e:
    raise e

[2025-07-06 13:57:16,138: INFO: common: yaml file config\config.yaml loaded successfully]
[2025-07-06 13:57:16,139: INFO: common: yaml file params.yaml loaded successfully]
[2025-07-06 13:57:16,141: INFO: common: Created Directory at : artifacts]
[2025-07-06 13:57:16,142: INFO: common: Created Directory at : artifacts/training]
[2025-07-06 13:57:16,385: INFO: 1776341136: Training started on device: cpu]
[2025-07-06 13:57:59,874: INFO: 1776341136: Epoch [1/10] - Total Loss: 6.9926 - Accuracy: 0.6774]
[2025-07-06 13:58:43,771: INFO: 1776341136: Epoch [2/10] - Total Loss: 5.5876 - Accuracy: 0.7823]
[2025-07-06 13:59:27,443: INFO: 1776341136: Epoch [3/10] - Total Loss: 4.2378 - Accuracy: 0.8602]
[2025-07-06 14:00:11,054: INFO: 1776341136: Epoch [4/10] - Total Loss: 3.8049 - Accuracy: 0.8844]
[2025-07-06 14:00:56,978: INFO: 1776341136: Epoch [5/10] - Total Loss: 3.3717 - Accuracy: 0.8898]
[2025-07-06 14:01:42,230: INFO: 1776341136: Epoch [6/10] - Total Loss: 3.8698 - Accuracy: 0.8737]
[2025